In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
%matplotlib inline
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
import keras
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.misc as misc
import time

In [ ]:
def decode_segmap(temp, plot=False):
    Sky = [128, 128, 128]
    Building = [128, 0, 0]
    Pole = [192, 192, 128]
    # Road_marking
    Road_marking = [255, 69, 0]
    Road = [128, 64, 128]
    Pavement = [60, 40, 222]
    Tree = [128, 128, 0]
    SignSymbol = [192, 128, 128]
    Fence = [64, 64, 128]
    Car = [64, 0, 128]
    Pedestrian = [64, 64, 0]
    Bicyclist = [0, 128, 192]
    Unlabelled = [0, 0, 0]

    label_colours = np.array([Sky, Building, Pole, Road, Pavement, Tree,
                              SignSymbol, Road_marking, Car,
                              Pedestrian, Fence, Unlabelled])

    height, width = temp.shape
    rgb = np.zeros((height, width, 3), dtype=np.uint8)

    for row in range(height):
        for col in range(width):
            label = temp[row, col]
            color = label_colours[label]
            #             if label==0:
            #                 print('label:', 0)
            rgb[row, col, :] = color

            #     rgb = rgb/255.0

    if plot:
        plt.imshow(rgb)
        plt.show()
    else:
        return rgb

In [ ]:
class vpn_mnist_config:
    def __init__(self):
        pass
    data_dir = os.path.join(os.path.expanduser('~/Data/mnist_test_seq.npy'))
    train_sequences_num = 7000
    truncated_steps = 9
    batch_size = 4
    num_timestamps = 10

In [ ]:
class GenerateMovingMnistData:
    def __init__(self):
        np.random.seed(123)
        sequences = np.load(vpn_mnist_config.data_dir).transpose((1, 0, 2, 3))
        sequences = np.expand_dims(np.squeeze(sequences), 4)
        shuffled_idxs = np.arange(sequences.shape[0])
        np.random.shuffle(shuffled_idxs)
        sequences = sequences[shuffled_idxs]

        self.num_timestamps = 10
        self.height = sequences.shape[2]
        self.width = sequences.shape[3]
        self.channel = sequences.shape[4]
        self.batch_size = vpn_mnist_config.batch_size
        self.seq_len = sequences.shape[0]
        self.train_len = vpn_mnist_config.train_sequences_num
        assert self.train_len < self.seq_len
        self.test_len = self.seq_len - self.train_len

        self.train_sequences = sequences[:self.train_len]
        self.test_sequences = sequences[self.train_len:]

    def next_train_batch_keras(self):
        while True:
            for i in range(0, self.train_len, self.batch_size):
                for j in range(i, i+self.batch_size):
                    idx = np.random.choice(self.train_len, self.batch_size)
                    current_sequence = self.train_sequences[idx]
                    current_sequence = current_sequence / 255.0
                # 输入前面10帧，以及预测的后面10帧
                prev_data = current_sequence[:, 0:self.num_timestamps]
                future_data = current_sequence[:, 1:self.num_timestamps+1]
                yield prev_data, future_data

    def next_test_batch_keras(self):
        while True:
            for i in range(0, self.test_len, self.batch_size):
                for j in range(i, i+self.batch_size):
                    idx = np.random.choice(self.test_len, self.batch_size)
                    current_sequence = self.test_sequences[idx]
                    current_sequence = current_sequence / 255.0
                # 输入前面10帧，以及预测的后面10帧
                prev_data = current_sequence[:, 0:self.num_timestamps]
                future_data = current_sequence[:, 1:self.num_timestamps+1]
                yield prev_data, future_data

    def test_batch(self):
        while True:
            idx = np.random.choice(self.test_sequences.shape[0], self.batch_size)
            current_sequence = self.test_sequences[idx]

            # 输入前面10帧，以及预测的后面10帧
            return current_sequence[:, 0:self.num_timestamps]

In [ ]:
class camvid_config:
    def __init__(self):
        pass
    data_dir = os.path.join(os.path.expanduser('~/Data/trainannot_moving_48x36.npy')) # 标注过的标签
#     data_dir = os.path.join(os.path.expanduser('~/Data/trainannot_moving_480x360.npy')) # 标注过的标签
#     data_dir = os.path.join(os.path.expanduser('~/Data/train_48x36.npy')) # 原图像
    train_sequences_num = 300
    train_sequences_num = 20
    truncated_steps = 9
    batch_size = 4
    num_timestamps = 10

In [ ]:
class GenerateCamVidData:
    def __init__(self):
        np.random.seed(123)
        sequences = np.load(camvid_config.data_dir).transpose((1, 0, 2, 3))
        sequences = np.expand_dims(np.squeeze(sequences), 4)
        shuffled_idxs = np.arange(sequences.shape[0])
        np.random.shuffle(shuffled_idxs)
        sequences = sequences[shuffled_idxs]

        self.num_timestamps = 10
        self.height = sequences.shape[2]
        self.width = sequences.shape[3]
        self.channel = sequences.shape[4]
        self.batch_size = camvid_config.batch_size
        self.seq_len = sequences.shape[0]
        self.train_len = camvid_config.train_sequences_num
        assert self.train_len < self.seq_len
        self.test_len = self.seq_len - self.train_len

        self.train_sequences = sequences[:self.train_len]
        self.test_sequences = sequences[self.train_len:]

    def next_train_batch_keras(self):
        while True:
            for i in range(0, self.train_len, self.batch_size):
                for j in range(i, i+self.batch_size):
                    idx = np.random.choice(self.train_len, self.batch_size)
                    current_sequence = self.train_sequences[idx]
                    current_sequence = current_sequence / 255.0
                # 输入前面10帧，以及预测的后面10帧
                prev_data = current_sequence[:, 0:self.num_timestamps]
                future_data = current_sequence[:, 1:self.num_timestamps+1]
                yield prev_data, future_data

    def next_test_batch_keras(self):
        while True:
            for i in range(0, self.test_len, self.batch_size):
                for j in range(i, i+self.batch_size):
                    idx = np.random.choice(self.test_len, self.batch_size)
                    current_sequence = self.test_sequences[idx]
                    current_sequence = current_sequence / 255.0
                # 输入前面10帧，以及预测的后面10帧
                prev_data = current_sequence[:, 0:self.num_timestamps]
                future_data = current_sequence[:, 1:self.num_timestamps+1]
                yield prev_data, future_data

    def test_batch(self):
        while True:
            idx = np.random.choice(self.test_sequences.shape[0], self.batch_size)
            current_sequence = self.test_sequences[idx]

            # 输入前面10帧，以及预测的后面10帧
            return current_sequence[:, 0:self.num_timestamps]

In [ ]:
# data_generate = GenerateMovingMnistData()
data_generate = GenerateCamVidData()

In [ ]:
n_frames = data_generate.num_timestamps
row = data_generate.height
col = data_generate.width
channel = data_generate.channel
batch_size = data_generate.batch_size

In [ ]:
model = Sequential()
model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3), input_shape=(n_frames, row, col, channel), padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3), padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3), padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3), padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(Conv3D(filters=1, kernel_size=(1, 3, 3), activation='sigmoid', padding='same', data_format='channels_last'))
model.compile(loss='mean_squared_error', optimizer='adadelta', metrics=['accuracy']) # mean_squared_error

In [ ]:
print(model.summary())

In [ ]:
# str(time.time())
# "keras_conv_lstm_weights_best{}.hdf5".format(str(time.time()))

In [ ]:
callbacks = []
model_checkpoint_filepath = "keras_conv_lstm_weights_best_{}.hdf5".format(str(time.time()))
# model_checkpoint_filepath = "keras_conv_lstm_weights_best.hdf5"
if os.path.exists(model_checkpoint_filepath):
    model.load_weights(model_checkpoint_filepath)
model_checkpoint = ModelCheckpoint(filepath=model_checkpoint_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks.append(model_checkpoint)

In [ ]:
model.fit_generator(generator=data_generate.next_train_batch_keras(), validation_data=data_generate.next_test_batch_keras(), steps_per_epoch=data_generate.train_len / batch_size, validation_steps=data_generate.test_len / batch_size, epochs=1000, verbose=1, callbacks=callbacks)

In [ ]:
# model.predict(np.array(data_generate.next_batch()))
data_sample_test = np.array(data_generate.test_batch())/255.0
data_sample_test_pred = model.predict(data_sample_test)

In [ ]:
data_sample_test_pred.shape

In [ ]:
for image_index in range(vpn_mnist_config.num_timestamps):
    img = data_sample_test[0, image_index, :, :, 0]
#     img = (img*255).astype(np.uint8)
#     img = decode_segmap(img)
    plt.subplot(4, 5, image_index+1)
    plt.imshow(img, cmap='gray')
    plt.axis('off')

In [ ]:
for image_index in range(vpn_mnist_config.num_timestamps):
    img = data_sample_test_pred[0, image_index, :, :, 0]
    plt.subplot(4, 5, image_index+1)
    plt.imshow(img, cmap='gray')
    plt.axis('off')

In [ ]:
for image_index in range(data_generate.num_timestamps):
    img = (data_sample_test_pred[0, image_index, :, :, 0]*255).astype(np.uint8)
    np.putmask(img, img >= 12, 8)
    img = decode_segmap(img)
    plt.subplot(4, 5, image_index+1)
    plt.imshow(img, cmap='gray')
    plt.axis('off')

In [ ]:
CamVidRoot = os.path.expanduser('~/Data/CamVid/')
CamVidRootTrainAnnot = os.path.join(CamVidRoot, 'trainannot_48x36')
CamVidRootTrainAnnotList = os.listdir(CamVidRootTrainAnnot)
CamVidRootTrainAnnotListLen = len(CamVidRootTrainAnnotList)
timestamps = data_generate.num_timestamps
for CamVidRootTrainAnnotListIndex in range(CamVidRootTrainAnnotListLen-timestamps):
#     print(CamVidRootTrainAnnotListIndex)
    # 20 timestamps
    moving_obj_data_sample = []
    for timestamp in range(timestamps):
        img = misc.imread(os.path.join(CamVidRootTrainAnnot, CamVidRootTrainAnnotList[CamVidRootTrainAnnotListIndex+timestamp]))
        height, width = img.shape
        mask_img_moving_mask = np.in1d(img, [8, 9]).reshape(height, width)
        img_moving = mask_img_moving_mask * img
#         plt.imshow(img_moving, cmap='gray')
        
        moving_obj_data_sample.append(img_moving)
        
    moving_obj_data_sample = np.array(moving_obj_data_sample)
    moving_obj_data_sample = moving_obj_data_sample[np.newaxis, :, :, :]
    moving_obj_data_sample = moving_obj_data_sample[:, :, :, :, np.newaxis]
    moving_obj_data_sample = moving_obj_data_sample/255.0
#     print('moving_obj_data_sample.shape:', moving_obj_data_sample.shape)
    moving_obj_data_sample_pred = model.predict(moving_obj_data_sample)
#     print('moving_obj_data_sample_pred.shape:', moving_obj_data_sample_pred.shape)
    moving_obj_data_sample_pred_next_frame = moving_obj_data_sample_pred[0, -1, :, :, 0]
    
    plt.clf()
    plt.subplot(121)
    img = decode_segmap(img)
    plt.imshow(img, cmap='gray')
    plt.subplot(122)
    plt.imshow(moving_obj_data_sample_pred_next_frame, cmap='hot')
    plt.show()
    time.sleep(1)
    if CamVidRootTrainAnnotListIndex==10:
        break